<a href="https://github.com/timeseriesAI/tsai-rs" target="_parent"><img src="https://img.shields.io/badge/tsai--rs-Time%20Series%20AI%20in%20Rust-blue" alt="tsai-rs"/></a>

# MiniRocket: A Very Fast Classifier for Time Series

This notebook demonstrates MiniRocket for time series classification and regression using **tsai-rs**.

Based on:
* Dempster, A., Schmidt, D. F., & Webb, G. I. (2020). **MINIROCKET: A Very Fast (Almost) Deterministic Transform for Time Series Classification**.

## What is MiniRocket?

MiniRocket is a time series classification method that is:
- **Extremely fast**: Much faster than deep learning methods
- **Highly accurate**: Competitive with state-of-the-art
- **Non-trainable features**: Uses fixed random kernels

### How it works:
1. Apply a large number (10,000) of fixed convolutions to the time series
2. Extract features (PPV - proportion of positive values) from each convolution
3. Train a simple linear classifier on these features

The key insight is that the convolution kernels don't need to be learned - carefully chosen fixed kernels work remarkably well.

## Install tsai-rs

```bash
cd crates/tsai_python
maturin develop --release
```

## Import Libraries

In [ ]:
import tsai_rs
import numpy as np
import time

print(f"tsai-rs version: {tsai_rs.version()}")
tsai_rs.my_setup()

## Load Data

In [ ]:
# Load multivariate dataset
dsid = 'NATOPS'
X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data(dsid, return_split=True)

n_vars = X_train.shape[1]
seq_len = X_train.shape[2]
n_classes = len(np.unique(y_train))

print(f"Dataset: {dsid}")
print(f"X_train shape: {X_train.shape}")
print(f"Variables: {n_vars}, Sequence length: {seq_len}, Classes: {n_classes}")

In [ ]:
# Standardize data
X_train_std = tsai_rs.ts_standardize(X_train.astype(np.float32), by_sample=True)
X_test_std = tsai_rs.ts_standardize(X_test.astype(np.float32), by_sample=True)

## MiniRocket Configuration

In [ ]:
# Basic MiniRocket configuration
minirocket_config = tsai_rs.MiniRocketConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes
)
print(f"MiniRocket config: {minirocket_config}")

In [ ]:
# MiniRocket with more features
minirocket_large = tsai_rs.MiniRocketConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes,
    n_features=10000  # Default is 10000
)
print(f"MiniRocket (large): {minirocket_large}")

## MiniRocket for Classification

In [ ]:
# Create TSDataset
train_ds = tsai_rs.TSDataset(X_train_std, y_train)
test_ds = tsai_rs.TSDataset(X_test_std, y_test)

print(f"Train dataset: {train_ds}")
print(f"Test dataset: {test_ds}")

In [ ]:
# Configure training for MiniRocket
# MiniRocket typically needs very few epochs since features are fixed
n_epochs = 10
batch_size = 64
lr = 3e-4

learner_config = tsai_rs.LearnerConfig(
    lr=lr,
    weight_decay=0.0,  # Often no weight decay for MiniRocket
    grad_clip=1.0
)

print(f"Training configuration:")
print(f"  Epochs: {n_epochs}")
print(f"  Batch size: {batch_size}")
print(f"  Learning rate: {lr}")

## MiniRocket for Regression

In [ ]:
# MiniRocket for regression (single output)
minirocket_reg = tsai_rs.MiniRocketConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=1  # Single output for regression
)
print(f"MiniRocket (regression): {minirocket_reg}")

In [ ]:
# Create regression targets (for demonstration)
y_train_reg = y_train.astype(np.float32)
y_test_reg = y_test.astype(np.float32)

train_ds_reg = tsai_rs.TSDataset(X_train_std, y_train_reg)
test_ds_reg = tsai_rs.TSDataset(X_test_std, y_test_reg)

print(f"Regression train dataset: {train_ds_reg}")

## Comparing MiniRocket with Other Architectures

MiniRocket is much faster than deep learning methods but achieves competitive accuracy.

In [ ]:
# Compare configurations
configs = {
    'MiniRocket': tsai_rs.MiniRocketConfig(
        n_vars=n_vars, seq_len=seq_len, n_classes=n_classes
    ),
    'InceptionTimePlus': tsai_rs.InceptionTimePlusConfig(
        n_vars=n_vars, seq_len=seq_len, n_classes=n_classes
    ),
    'ResNetPlus': tsai_rs.ResNetPlusConfig(
        n_vars=n_vars, seq_len=seq_len, n_classes=n_classes
    ),
    'TST': tsai_rs.TSTConfig(
        n_vars=n_vars, seq_len=seq_len, n_classes=n_classes
    ),
}

print(f"{'Architecture':<20} {'Type':<15} {'Typical Training Time'}")
print("-" * 60)

training_times = {
    'MiniRocket': 'Very Fast (<1 min)',
    'InceptionTimePlus': 'Medium (5-20 min)',
    'ResNetPlus': 'Medium (5-20 min)',
    'TST': 'Slow (10-60 min)',
}

types = {
    'MiniRocket': 'Feature-based',
    'InceptionTimePlus': 'CNN',
    'ResNetPlus': 'CNN',
    'TST': 'Transformer',
}

for name in configs.keys():
    print(f"{name:<20} {types[name]:<15} {training_times[name]}")

## When to Use MiniRocket

| Scenario | Recommendation |
|----------|---------------|
| Quick baseline | MiniRocket |
| Limited compute | MiniRocket |
| Small dataset (<1000) | MiniRocket |
| Need fast inference | MiniRocket |
| Complex patterns | Deep learning (InceptionTime, TST) |
| Large dataset | Deep learning with augmentation |

## Using MiniRocket with Different Datasets

In [ ]:
# Test MiniRocket configuration on different datasets
datasets = ['ECG200', 'GunPoint', 'FordA', 'Wafer', 'NATOPS']

print(f"{'Dataset':<15} {'Vars':<6} {'Len':<8} {'Classes':<8} {'Train':<8}")
print("-" * 50)

for dsid in datasets:
    try:
        X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data(dsid, return_split=True)
        n_vars = X_train.shape[1]
        seq_len = X_train.shape[2]
        n_classes = len(np.unique(y_train))
        n_train = X_train.shape[0]
        
        # Create config
        config = tsai_rs.MiniRocketConfig(
            n_vars=n_vars, seq_len=seq_len, n_classes=n_classes
        )
        
        print(f"{dsid:<15} {n_vars:<6} {seq_len:<8} {n_classes:<8} {n_train:<8}")
    except Exception as e:
        print(f"{dsid:<15} Error: {e}")

## Data Augmentation with MiniRocket

While MiniRocket uses fixed kernels, you can still use data augmentation on the input time series.

In [ ]:
# Load data
dsid = 'NATOPS'
X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data(dsid, return_split=True)

# Standardize
X_train_std = tsai_rs.ts_standardize(X_train.astype(np.float32), by_sample=True)

# Apply augmentation
X_train_aug = tsai_rs.add_gaussian_noise(X_train_std, std=0.05, seed=42)
X_train_aug = tsai_rs.mag_scale(X_train_aug, scale_range=(0.9, 1.1), seed=42)

print(f"Original data shape: {X_train_std.shape}")
print(f"Augmented data shape: {X_train_aug.shape}")

## Complete MiniRocket Pipeline

In [ ]:
# Complete pipeline example
def minirocket_pipeline(dsid):
    """Complete MiniRocket classification pipeline."""
    
    # 1. Load data
    print(f"Loading dataset: {dsid}")
    X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data(dsid, return_split=True)
    
    # 2. Get dimensions
    n_vars = X_train.shape[1]
    seq_len = X_train.shape[2]
    n_classes = len(np.unique(y_train))
    
    print(f"  Shape: {X_train.shape}")
    print(f"  Variables: {n_vars}, Length: {seq_len}, Classes: {n_classes}")
    
    # 3. Standardize
    X_train_std = tsai_rs.ts_standardize(X_train.astype(np.float32), by_sample=True)
    X_test_std = tsai_rs.ts_standardize(X_test.astype(np.float32), by_sample=True)
    
    # 4. Configure MiniRocket
    config = tsai_rs.MiniRocketConfig(
        n_vars=n_vars,
        seq_len=seq_len,
        n_classes=n_classes,
        n_features=10000
    )
    print(f"  Config: {config}")
    
    # 5. Create datasets
    train_ds = tsai_rs.TSDataset(X_train_std, y_train)
    test_ds = tsai_rs.TSDataset(X_test_std, y_test)
    
    # 6. Configure training
    learner_config = tsai_rs.LearnerConfig(
        lr=3e-4,
        weight_decay=0.0,
        grad_clip=1.0
    )
    
    print(f"  Ready for training!")
    
    return config, train_ds, test_ds, learner_config

# Run pipeline
config, train_ds, test_ds, learner_config = minirocket_pipeline('NATOPS')

## Summary

This notebook demonstrated MiniRocket for time series classification:

### Key Characteristics
- **Fast**: Uses fixed convolution kernels (no training needed for features)
- **Accurate**: Competitive with deep learning methods
- **Simple**: Only trains a linear classifier on features

### Configuration
```python
config = tsai_rs.MiniRocketConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes,
    n_features=10000  # Number of features to extract
)
```

### When to Use
- Quick baseline experiments
- Limited computational resources
- Small datasets
- Need for fast inference

### Limitations
- Fixed features may not capture all patterns
- May underperform on very complex datasets
- Less interpretable than some methods

In [ ]:
# Quick reference
print("MiniRocket Quick Reference")
print("=" * 50)
print("\n# Configuration")
print("config = tsai_rs.MiniRocketConfig(")
print("    n_vars=n_vars,")
print("    seq_len=seq_len,")
print("    n_classes=n_classes,")
print("    n_features=10000  # Optional, default 10000")
print(")")

print("\n# Training config (few epochs needed)")
print("learner_config = tsai_rs.LearnerConfig(")
print("    lr=3e-4,")
print("    weight_decay=0.0")
print(")")